A aplicação simul-shop

O um simul-shop é um simulador de loja online.

Inspirado no micro-sock, desenvolvida pela [weaveworks](https://www.weave.works/) e [distribuída](https://github.com/microservices-demo/microservices-demo/tree/master) de forma gratuíta (licença [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0)) para demonstrar seu produto [Waeve Scope](https://www.weave.works/oss/scope/).

O simulador foi construída usando [Python](https://www.python.org/) e é empacotado uma imagem Docker e entregue em um cluster de kubernetes.

## Simul Shop implementação

O simulador não tem uma interface web é apenas um programa em python que recebe requisições e invoca outras URLs com base nos parâmetros passados por variável de ambiente.

Por exemplo, vamos simular uma aplicação que tem um front-end e um backend, usando apenas o docker podemos executar da seguinte forma:

![](media/simul-shop-fb.png)

```bash
# Create net
docker network create my-net

# backend
docker run -d --rm \
--hostname backend \
--network my-net \
--name backend \
-e SCHED_CALL_URL_LST=http://localhost:8000/healthz \
-e SCHED_CALL_INTERVAL=5 \
kdop/generic-service:0.0.1

# front-end
docker run -d --rm \
    --network my-net \
    --hostname front-end \
    --name front-end \
    -e SCHED_CALL_URL_LST=http://front-end:8000/s \
    -e SCHED_CALL_INTERVAL=10 \
    -e SPLIT_CALL_URL_LST=http://backend:8000 \
    kdop/generic-service:0.0.1
```

Vamos acompanhar os _logs_ para ver o que está acontecento.

```bash
# Terminal 1
docker logs -f front-end
# Terminal 2
docker logs -f backend
```

Para remover os recursos

```bash
# Parando e excluindo os containers
docker kill front-end backend
# Clean-up
docker network rm my-net
```

### Istio + Simul Shop

Configuramos o simul-shop para criar uma malha de serviços, sua arquittura é:

![arquitetura](media/simul-shop-arq.png)

Iremos utiliza-lo para explorar os recursos do Istio.

## Instalando a aplicação

Com o clone do repo istio-curso, você tem um diretório com a aplicação.

In [2]:
ls -la exemplos/simul-shop-manifests

total 24
drwxr-xr-x   4 anderson  staff   128 Oct 22 14:34 .
drwxr-xr-x  10 anderson  staff   320 Oct 22 14:36 ..
-rw-r--r--   1 anderson  staff  5607 Oct 22 14:35 deployments.yaml
-rw-r--r--   1 anderson  staff  2638 Oct 22 14:34 services.yaml


A configuração para kubernetes compreende apenas dois recursos, o _deployment_ e o _service_ para cada um dos módulos da nossa aplicação, de acordo coma arquitetura da seção anterior.

* [exemplos/simul-shop-manifests/deployments.yaml](exemplos/simul-shop-manifests/deployments.yaml)
* [exemplos/simul-shop-manifests/services.yaml](exemplos/simul-shop-manifests/services.yaml)

Como na aplicação de demo, criamos um deployment para a imagem `kdop/generic-service 0.0.1` e adiconamos os _labels_ para representar o nome do módulo e sua versão.

```yaml
labels:
    app: "nome-do-modulo"
    version: "versao"
```

Os serviços foram configurados para acessar cada um dos módulos:

```yaml
metadata:
  name: front-end # <--- Cria um DNS com nome nome-modulo.
...
spec:
  type: ClusterIP
  selector:
    app: front-end # <--- E aponta para um POD que tennha o label nome-modulo.
```

Dessa forma os serviços podem ser endereçados na forma http://nome-modulo:porta/. Na nossa arquitetura:

* http://front-end:80000
* http://login:80000
* http://catalogue:80000
* http://orders:80000
* http://shipping:80000
* http://cart:80000
* http://payment:80000
* http://accounts:80000
* http://orders-db:80000
* http://queue:80000
* http://cart-db:80000
* http://accounts-db:80000




Vamos aplicar a configuração no cluster:

In [5]:
# Acesso ao cluster
export KUBECONFIG=~/.kube/config 

In [6]:
kubectl apply -f exemplos/simul-shop-manifests/

deployment.apps/front-end created
deployment.apps/login created
deployment.apps/catalogue created
deployment.apps/orders created
deployment.apps/orders-db created
deployment.apps/shipping created
deployment.apps/queue created
deployment.apps/cart created
deployment.apps/cart-db created
deployment.apps/payment created
deployment.apps/accounts created
deployment.apps/accounts-db created
service/front-end created
service/login created
service/catalogue created
service/accounts created
service/accounts-db created
service/cart created
service/cart-db created
service/orders created
service/orders-db created
service/shipping created
service/queue created
service/payment created


Vamos verificar o que foi criado:

In [11]:
kubectl get all

NAME                              READY   STATUS            RESTARTS   AGE
pod/accounts-6576c98567-276qs     0/2     PodInitializing   0          52s
pod/accounts-db-c66f6d886-pfx7k   0/2     PodInitializing   0          52s
pod/cart-755d886f8f-cz62t         0/2     PodInitializing   0          54s
pod/cart-db-768bd6f7cd-n7mtk      0/2     PodInitializing   0          53s
pod/catalogue-6ccc68b854-rcb29    2/2     Running           0          55s
pod/front-end-5fdddf4dcb-vl8v9    2/2     Running           0          55s
pod/login-868764c74b-8znjz        1/2     Running           0          55s
pod/orders-77b45cc699-6r6m2       0/2     PodInitializing   0          55s
pod/orders-db-59f88cf7b-d28rm     2/2     Running           0          55s
pod/payment-8c5f4c94d-8vnlx       0/2     PodInitializing   0          53s
pod/queue-6884748c5d-2qmst        0/2     PodInitializing   0          54s
pod/shipping-b7b68d8d9-d78p6      2/2     Running           0          54s

NAME                  TY

Você pode repetir o comando algumas vezes para ver a situaçào dos PODs e se estiver utilizando um terminal pode adicionar o comando `watch kubectl get all` para que atualize automaticamente.

Após alguns minutos todos os PODs devem estar na seguinte situação:

```bash
accounts-6576c98567-276qs     2/2     Running   0          3m14s
accounts-db-c66f6d886-pfx7k   2/2     Running   0          3m14s
cart-755d886f8f-cz62t         2/2     Running   0          3m16s
cart-db-768bd6f7cd-n7mtk      2/2     Running   0          3m15s
catalogue-6ccc68b854-rcb29    2/2     Running   0          3m17s
front-end-5fdddf4dcb-vl8v9    2/2     Running   0          3m17s
login-868764c74b-8znjz        2/2     Running   0          3m17s
orders-77b45cc699-6r6m2       2/2     Running   0          3m17s
orders-db-59f88cf7b-d28rm     2/2     Running   0          3m17s
payment-8c5f4c94d-8vnlx       2/2     Running   0          3m15s
queue-6884748c5d-2qmst        2/2     Running   0          3m16s
shipping-b7b68d8d9-d78p6      2/2     Running   0          3m16s
```

Dependendo da quantidade de recursos disponíveis para o cluster de kubernetes o tempo pode variar e alguns PODs podem falhar (CrashBackoff) algumas vezes (RESTARTS > 0).

Para conhecer mais sobre o ciclo de vida de um POD, acesse [Kubernetes - POD Lifecycle](https://kubernetes.io/docs/concepts/workloads/pods/pod-lifecycle/)

Se algum dos seus PODs não alcançar o estágio _Running_ verifique o motivo com o comando `kubectl describe pod -l app=<nome-do-modulo>`, por exemplo:

In [12]:
kubectl describe pod -l app=front-end

Name:         front-end-5fdddf4dcb-vl8v9
Namespace:    default
Priority:     0
Node:         docker-desktop/192.168.65.3
Start Time:   Fri, 23 Oct 2020 08:25:04 -0300
Labels:       app=front-end
              istio.io/rev=default
              pod-template-hash=5fdddf4dcb
              security.istio.io/tlsMode=istio
              service.istio.io/canonical-name=front-end
              service.istio.io/canonical-revision=1.0
              version=1.0
Annotations:  prometheus.io/path: /stats/prometheus
              prometheus.io/port: 15020
              prometheus.io/scrape: true
              sidecar.istio.io/status:
                {"version":"8e6e902b765af607513b28d284940ee1421e9a0d07698741693b2663c7161c11","initContainers":["istio-init"],"containers":["istio-proxy"]...
Status:       Running
IP:           10.1.0.123
IPs:
  IP:           10.1.0.123
Controlled By:  ReplicaSet/front-end-5fdddf4dcb
Init Containers:
  istio-init:
    Container ID:  docker://9cd359e12db2be9594f8c3398dd50

Se você for direto para o final, verá uma seção _Events_, e uma série de eventos que aconteceram com o seu POD, os últimos eventos indicaram algum tipo de problema, como por exemplo, que não foi possível encontrar um nó para fazer o _deploy_, o que indica que você precisa aumentar os recursos para o kuberntes (escala vertical), dando mais memória e/ou CPU. Isso pode ser feito no painel do docker-desktop em configurações, para mais informações acesse [docker-desktop - Resources](https://docs.docker.com/docker-for-windows/#resources)

## Istio proxy

Vamos olhar novamente o describe do POD front-end que acabamos de executar, você deve ter percebido algo diferente, há muita informação que não configuramos e quando listamos os PODs (`kubectl get pods`) havia uma indicação que 2/2 containers estavam prontos (READY), mas nossa aplicação tem apenas um, o segundo container pode ser descoberto procurando no describe a seção `Containers:`.

Neste _describe_ encontramos dois _containers_:

```bash
# no início da seção
Containers:
  front-end:
    Container ID:   docker://0640db5071daddccdb883410508475cf1f6c447dabc1a841b60a8a1de74683d3
    Image:          kdop/generic-service:0.0.1
# mais abaixo
  istio-proxy:
    Container ID:  docker://9859fd6e75d4cdf9ea7f26be9e3f8fc47a6cf79664d400b87259652113792a30
    Image:         docker.io/istio/proxyv2:1.7.3
```

Um dos container utiliza a imagem da nossa aplicação e o outro do proxy do Istio. Esse container foi adicionado automaticamente pelo `istiod` quando instalamos nossa aplicação no _namespace_ `default`, porque esse _namespace_ está com um rótulo (`istio-injection=enabled`) que indica para o `istiod` executar essa operação de injeção de um _sidecar_ no nosso POD.

In [13]:
kubectl describe ns default

Name:         default
Labels:       istio-injection=enabled
Annotations:  <none>
Status:       Active

No resource quota.

No LimitRange resource.


Aqui uma recapitulação rápida sobre o padrão _sidecar_ utilizado pelo Istio, para saber sobre outros padrões, acesse o curso [Kubernetes avançado para iniciantes](TODO).

### O padrão _sidecar_

!TODO

[Kubernetes — Learn Sidecar Container Pattern](https://medium.com/bb-tutorials-and-thoughts/kubernetes-learn-sidecar-container-pattern-6d8c21f873d

Agora que nossa aplicação está instalada, na próxima seção vamos visualizá-la no kiali e verificar se confere com nossa arquitetura.